# Notebook for Predicting WFQC

Import the modules

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import glob
import os
import wave
import pylab
#from matplotlib import plt
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import multiprocessing
import gc
import random
from obspy.core import read, UTCDateTime

Directories setup

In [2]:
current_dir = "/home/roberto/notebooks/WFQCMODEL/"
data_dir = current_dir + "Data/"
seism_dir = '/home/roberto/notebooks/DATAFIL/'
output_spectrogram_dir = data_dir + "Extracted_Spectrogram_Full_Analysis/" 


In [3]:
if not os.path.exists(output_spectrogram_dir):
    os.makedirs(output_spectrogram_dir)


In [4]:
seis_filenames = glob.glob(seism_dir + '/????')

Preparing the DataFrame

In [5]:
new_list=[]
i=1
for subs in seis_filenames:
    new2=os.path.basename(subs)
    velfil=glob.glob(subs+'/*.vel')
    for file in velfil:
        sub=os.path.basename(file)
        cont=os.path.dirname(file)
#
        last=os.path.basename(cont)
        new_list.append([i,sub,last])
        i=i+1

In [6]:
df = pd.DataFrame(new_list, columns =['Num', 'Waveform','Sub']) 

Plot a spectrogra using OBSPY

In [7]:
def graph_spectrogram(wav_file, sub, serial):
#    fig=pyplot.figure(num=None, figsize=(19, 12))   
#    ax = pyplot.axes()
#    ax.set_axis_off()
    st = read(wav_file)
    tr=st[0]
    tr.normalize()
    fig = plt.figure()
    #ax = plt.Axes(fig,[0.,0.,.8,.8])
    ax = plt.Axes(fig,[0.1, 0.1, 0.7, 0.6])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.set_axis_off() 
    begin_TimeStamp=tr.stats.starttime
    audio_begin_TimeStamp=str(begin_TimeStamp.year)+'-'+str(begin_TimeStamp.month)+'-'+str(begin_TimeStamp.day)+'T'+str(begin_TimeStamp.hour)+'-'+str(begin_TimeStamp.minute)
    start_second=str(begin_TimeStamp.second)
    filename=output_spectrogram_dir + serial + '-' + audio_begin_TimeStamp + '-' + str(start_second)  + '.png'
#    print(filename)
    fig = tr.spectrogram(show=False,axes=ax)
#   # filename=output_spectrogram_dir+"AAA"+".png"
    plt.savefig(filename)
    plt.close()
 


In [8]:
def generate_spectrogram(i):
    try:
        Filenam = df.loc[i, 'Waveform'] 
        Sub = df.loc[i, 'Sub']
        Fileall=seism_dir+Sub+'/'+Filenam
        serial,comp,ty=Filenam.split('.')
        serial=serial+'_'+comp+'_'+Sub+'_' 
#        print([serial,Sub,Fileall])
        return graph_spectrogram(Fileall,Sub,serial)
    except:
        pass

Parallel multiprocess for creating spectrograms in the Output_spectrogram folder

In [9]:
num_cores = multiprocessing.cpu_count()

In [10]:
output_spectrogram_dir = data_dir + "Extracted_Spectrogram_Full_Analysis/"

In [11]:
spectrograms = Parallel(n_jobs=num_cores)(delayed(generate_spectrogram)(i) for i in (range(len(df))))